# Consistency check with $T_{\rm{X}}$ and $L_{\rm{X}}$ measured from Chandra data

## Import Statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity
import os
from shutil import rmtree
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter

from ciao_contrib.runtool import specextract

import xga
# This just sets the number of cores this analysis is allowed to use - IRRELEVANT HERE REALLY AS XGA DOESN'T SUPPORT CHANDRA YET
xga.NUM_CORES = 20
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/lum_temp_rad_pipeline')[0]+'xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
    
%matplotlib inline

Assembling list of ObsIDs: 100%|██████████| 8/8 [00:00<00:00, 19.73it/s]


## Setting up directories

Need to make sure that some of the directories that we will be storing output files and figures in actually exist:

In [2]:
# if not os.path.exists('../../outputs/results/'):
#     os.makedirs('../../outputs/results/')
    
# if not os.path.exists('../../outputs/figures/gm_prop_comparisons/'):
#     os.makedirs('../../outputs/figures/gm_prop_comparisons/')

## Reading the XGA sample/results files from XMM analysis

We read in the files generated by our various runs of the XGA-LTR in another notebook in this repository. They contain the radii we measured, as well as other useful information - we're going to use Chandra to measure properties within the XMM defined radii (for the limited groups that have Chandra coverage):

In [3]:
r500_samp = pd.read_csv('../../outputs/results/ltr_r500_pipeline_results.csv')
r500_froztemp_samp = pd.read_csv('../../outputs/results/ltr_r500_froztemp_pipeline_results.csv')

r2500_samp = pd.read_csv('../../outputs/results/ltr_r2500_pipeline_results.csv')
r2500_froztemp_samp = pd.read_csv('../../outputs/results/ltr_r2500_froztemp_pipeline_results.csv')

As previously stated, few of our groups have usable Chandra coverage, so we'll cut down the results files to leave only the relevant entries:

In [4]:
with_ch_names = ['12833A', '12833B', 'J215504.8-093520', 'J215437.2-091532.4']

r500_samp = r500_samp[r500_samp['name'].isin(with_ch_names)]
r500_froztemp_samp = r500_froztemp_samp[r500_froztemp_samp['name'].isin(with_ch_names)]

r2500_samp = r2500_samp[r2500_samp['name'].isin(with_ch_names)]
r2500_froztemp_samp = r2500_froztemp_samp[r2500_froztemp_samp['name'].isin(with_ch_names)]

As an illustration of the contents of one of these files:

In [5]:
r500_samp

,name,ra,dec,redshift,r500,r500+-,Tx500,Tx500-,Tx500+,Lx500_0.5-2.0,Lx500_0.5-2.0-,Lx500_0.5-2.0+,Lx500_0.01-100.0,Lx500_0.01-100.0-,Lx500_0.01-100.0+,Tx500ce,Tx500ce-,Tx500ce+,Lx500ce_0.5-2.0,Lx500ce_0.5-2.0-,Lx500ce_0.5-2.0+,Lx500ce_0.01-100.0,Lx500ce_0.01-100.0-,Lx500ce_0.01-100.0+
9,12833A,129.49680,44.248700,0.145,484.161429,22.267250,1.394590,0.098024,0.113226,3.419655e+42,4.326240e+41,2.991426e+41,6.828256e+42,9.969550e+41,7.096560e+41,1.416080,0.145301,0.239000,2.240773e+42,3.645432e+41,2.770394e+41,4.498901e+42,7.032202e+41,5.884023e+41
10,12833B,129.44500,44.379000,0.145,435.477156,26.364544,1.161630,0.135227,0.116765,2.511945e+42,2.677573e+41,3.283677e+41,4.654556e+42,7.296128e+41,8.407739e+41,0.926943,0.120020,0.134047,1.378771e+42,3.555113e+41,3.233896e+41,2.309754e+42,5.456964e+41,3.761557e+41
13,J215504.8-093520,328.76992,-9.588805,0.077,581.016969,20.416613,1.727820,0.077037,0.089383,7.789182e+42,2.873907e+41,4.574905e+41,1.769054e+43,1.037285e+42,8.310896e+41,1.708510,0.101667,0.138426,4.889413e+42,3.539555e+41,4.222563e+41,1.106974e+43,9.531763e+41,8.356995e+41
14,J215437.2-091532.4,328.65500,-9.259000,0.078,393.970353,18.283530,0.350344,0.021447,0.024195,4.768932e+40,4.768932e+40,5.223045e+43,8.846684e+40,8.846684e+40,7.043529e+43,0.335890,0.016225,0.023720,1.039277e+43,4.677879e+41,4.965552e+41,2.661767e+43,1.319874e+42,1.311632e+42


## Setting up an XGA ClusterSample

Unlike our overall XMM analysis, we will not be using XGA to measure temperatures and luminosities from Chandra observations - this is because Chandra support is still early in development. We will still declare a ClusterSample however, simply because it will extract nH values for us, and may help set up some of the arguments for Chandra specextract and the eventual XSPEC fits:

In [6]:
srcs = ClusterSample(r500_samp['ra'].values, r500_samp['dec'].values, r500_samp['redshift'].values, 
                     r500_samp['name'].values, r500=Quantity(r500_samp['r500'].values, 'kpc'), 
                     use_peak=False, load_fits=True, clean_obs=False)
srcs.info()
srcs.suppressed_warnings

Setting up Galaxy Clusters: 100%|██████████| 4/4 [00:05<00:00,  1.48s/it]


-----------------------------------------------------
Number of Sources - 4
Redshift Information - True
Sources with ≥1 detection - 4 [100%]
-----------------------------------------------------




/mnt/home/turne540/software/anaconda3/envs/chandra_daxa_dev/lib/python3.11/site-packages/xga/samples/extended.py:279: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()


{'J215504.8-093520': ["J215504.8-093520 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data"],
 'J215437.2-091532.4': ['A PSF sized extended source has been detected in 0204310101 and is very close to the user supplied coordinates of J215437.2-091532.4. It will not be excluded from analysis due to the possibility of a mis-identified cool core']}

## Defining which Chandra observations are relevant for each group

As we don't have an XGA ClusterSample to do this for us, we previously examined the Chandra obserations and identified which are relevant to each of the groups - here this information is placed in a dictionary so that it can be accessed programmatically: 

In [7]:
rel_ch_obs = {'12833A': ['15378'], 
              '12833B': ['15378'], 
              'J215504.8-093520': ['2957', '2958'], 
              'J215437.2-091532.4': ['2957', '2958']}

Temporary notes of what specextract is going to want:

specextract  infile outroot [bkgfile] [asp] [dtffile] [mskfile]
[rmffile] [badpixfile] [dafile] [bkgresp] [weight] [weight_rmf]
[resp_pos] [refcoord] [correctpsf] [combine] [readout_streakspec]
[grouptype] [binspec] [bkg_grouptype] [bkg_binspec] [energy] [channel]
[energy_wmap] [binarfcorr] [binwmap] [binarfwmap] [parallel] [nproc]
[tmpdir] [clobber] [verbose]


## Generating Chandra spectra

### Setting up useful values

In [18]:
evt_pth = "../../data/archives/OVIGroups/processed_data/chandra/{oi}/events/obsid{oi}-instACIS-subexpE001-en-cleanevents.fits"

r500_inn_bck_fac = 1.05
r500_out_bck_fac = 1.5

In [19]:
for src_name in rel_ch_obs:

    rel_src = srcs[src_name]
    cur_r500_deg = rel_src.get_radius('r500', 'deg')

    cur_r500_bck_inn = cur_r500_deg * r500_inn_bck_fac
    cur_r500_bck_out = cur_r500_deg * r500_out_bck_fac

    reg_str = "CIR({ra}d,{dec}d,{r}d)".format(ra=rel_src.ra_dec[0].value, dec=rel_src.ra_dec[1].value, r=cur_r500_deg.value)
    print(reg_str)
    
    rel_ois = rel_ch_obs[src_name]
    for oi in rel_ois:
        rel_evt = evt_pth.format(oi=oi)
        print(rel_evt)
        stop

CIR(129.4968d,44.2487d,0.05291941545991717d)
../../data/archives/OVIGroups/processed_data/chandra/15378/events/obsid15378-instACIS-subexpE001-en-cleanevents.fits


NameError: name 'stop' is not defined

In [11]:
rel_src.nH

<Quantity 0.0414 1e+22 / cm2>